In [20]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/maffsojah/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random', 'add']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [49]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('Bank-Customer-Classifier') \
    .getOrCreate()
    
sc = pyspark.SparkContext

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
import math
from operator import add
import datetime

In [60]:
def load_training_set():
    X = np.load("../datasets/X.npy")
    y = np.load("../datasets/y.npy")
    dataset = []
    for i in range(y.shape[0]):
        x_vector = X[i].reshape((-1, 1))
        y_vector = np.eye(10)[y[i]-1].reshape((-1, 1))
        dataset.append((x_vector, y_vector))
        
    return dataset    

In [69]:
# parallelize data into spark nodes.

def parse_data():
    dataset = load_training_set()
    data_set = sc.parallelize(dataset)
    #split it into training, validation and test sets. Use the randomSplit method
    weights = [.8, .1, .1]
    seed = 8888
    train_set, validate_set, test_set = data_set.randomSplit(weights, seed)
    
    return (train_set, validate_set, test_set)

In [62]:
# derivative of the sigmoid functionin terms of the output
def dsigmoid(y):
    gz = sigmoid(y)
    return gz * (1.0 - gz)

def sigmoid(x):
    # exp function provided by numpy can support vector operation by default
    return 1.0 / (1.0 + np.exp(-x))


In [63]:
# the class is the NN
class NN:
    def __init__(self, ni, nh, no):
        # inputs, hidden, and output nodes
        self.n1 = ni + 1 # +1 for bias node
        self.n2 = nh
        self.n3 = no
        
        # weights variables (theta) in the model
        self.w1 = self.weights_init(ni, nh)
        self.w2 = self.weights_initi(nh, no)
        
        # accumulating the gradient from all the train samples
        self.Delta1 = np.zeros(shape=(self.n2, self.n1)) # for w1
        self.Delta2 = np.zeros(shape=(self.n3, self.n2+1)) # for w2
        
        def weights_init(self, l_in, l_out):
            eps_init = 0.12
            ret = mp.random.rand(l_out, 1+l_in) * 2 * eps_init - eps_init
            return ret
        

In [64]:
# function for forward and back propagation and returning all Deltas 
# as well as prediction error for display.
def ann_train_eval(w, sample):
    w1, w2 = w
    x = sample[0]
    y = sample[1]
    a1 = np.vstack(([1.0], x))
    
    # hidden activations
    z2 = np.dot(w1, a1)
    a2 = np.vstack(([1.0], sigmoid(z2)))
    
    # output activations
    z3 = np.dot(w2, a2)
    a3 = sigmoid(z3)
    
    # starting backpropagation and calculating error terms for output
    delta3 = a3 - y
    
    # skipping first column which is for bias
    delta2 = (np.dot(w2.T, delta3))[1:] * dsigmoid(z2)
    
    # gradient from all train samples for accumulating
    Delta1 = np.dot(delta2, a1.T)
    Delta2 = np.dot(delta3, a2.T)
    
    # mse for calculating train predict error, mse was used, just to show the minimization, 
    # if using other optimization method, should use cost function
    
    return np.array([Delta1, Delta2, np.mean((a3 - y)**2)])
    

In [65]:
# this function is provided to do prediction by using a trained neural network model
def nn_predict(ann, inputs):
    x = inputs.reshape((-1, 1))
    a1 = np.vstack(([1.0], x))

    # hidden activations
    z2 = np.dot(ann.w1, a1)
    a2 = np.vstack(([1.0], sigmoid(z2)))

    # output activations
    z3 = np.dot(ann.w2, a2)
    a3 = sigmoid(z3)
    return a3

In [66]:
def nn_train(ann, train_set, max_iter):
    Lambda = 10.01 # Regularization parameter (to avoid overfit issue)
    m = train_set.count()
    for iteration in range(max_iter):
        eval_res = train_set.map(lambda x: ann_train_eval((ann.w1, ann.w2), x))
        
        # calculate derivation of weights via average bp results
        average_eval = eval_res.reduce(add) / train_set.count()
        dw1 = average_eval[0]
        dw2 = average_eval[1]
        
        # mean error sololy for display
        mean_err = average_eval[2]
        
        # for all the weights, you should not apply regulization on the first column since they are for bias
        for i in range(ann.w1.shape[0]):
            ann.w1[i, 0] = ann.w1[i, 0] - dw1[i, 0]
        for i in range(ann.w1.shape[0]):
            for j in range(1, ann.w1.shape[1]):
                # here learn rate is 1.0, Lambda is the Regularization parameter
                ann.w1[i, j] = ann.w1[i, j] - (dw1[i, j] + (Lambda/m) * ann.w1[i, j])
        
        for i in range(ann.w2.shape[0]):
            ann.w2[i, 0] = ann.w2[i, 0] - dw2[i, 0]
        for i in range(ann.w2.shape[0]):
            for j in range(1, ann.w2.shape[1]):
                ann.w2[i, j] = ann.w2[i, j] - (dw2[i, j] + (Lambda/m) * ann.w2[i, j])
        if 0 == iteration % 50:
            print ("mean error p", mean_err)
    return ann

In [70]:
# load and parse data, randomly devide dataset to three parts for train, validate and test separately 
train_set, validate_set, test_set = parse_data()
#print train_set.count(), validate_set.count(), test_set.count()

n = NN(400, 50, 10)

# train it with some patterns
dt_st = datetime.datetime.now()
print ("train start at: {0}".format(dt_st))

nt = nn_train(n, train_set, 400)

dt_end = datetime.datetime.now()
print ("train end at: {0}".format(dt_end))
print ("time elapse in traininf: {0} seconds".format((dt_end-dt_st).seconds))

TypeError: unbound method parallelize() must be called with SparkContext instance as first argument (got list instance instead)

In [50]:
# test trained neural network model on train dataset
n_set = train_set.count()
val_res = train_set.map(lambda x : 1 + np.argmax(nn_predict(nt, x[0]))).collect()
actual_res = train_set.map(lambda x : 1 + np.argmax(x[1])).collect()

accurate = 0
for idx in range(n_set):
    if val_res[idx] == actual_res[idx]:
        accurate += 1
print("train set accuracy: {0} %".format(100.0 * accurate / n_set))
    

NameError: name 'train_set' is not defined

In [ ]:
# test trained neural network model on validation data set
n_val_set = validate_set.count()
val_res = validate_set.map(lambda x : 1 + np.argmax(nn_predict(nt, x[0]))).collect()
actual_res = validate_set.map(lambda x : 1 + np.argmax(x[1])).collect()

accurate = 0
for idx in range(n_val_set):
    if val_res[idx] == actual_res[idx]:
        accurate += 1
print("validation set accuracy: {0} %".format(100.0 * accurate / n_val_set))
    

In [ ]:
# test trained neural network model on validation data set
n_val_set = test_set.count()

val_res = test_set.map(lambda x : 1 + np.argmax(nn_predict(nt, x[0]))).collect()
actual_res = test_set.map(lambda x : 1 + np.argmax(x[1])).collect()

accurate = 0
for idx in range(n_val_set):
    if val_res[idx] == actual_res[idx]:
        accurate += 1
print("test set accuracy: {0} %".format(100.0 * accurate / n_val_set))

In [4]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Loading the data

data = spark.read.format('csv').load('../datasets/dummy.csv')


# splitting the data
splits = data.randomSplit([1])
train = splits[0]

NameError: name 'spark' is not defined

In [2]:
# 8 input layers(features), 9 and 8 hidden layers and 3 output classes

layers = [8, 9, 8, 3]

In [3]:
# trainer with parameters

trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128)


# training the model
model = trainer.fit()

NameError: name 'MultilayerPerceptronClassifier' is not defined